# Build your first FREE AI agent with Ollama and Strands

---

[![5aharsh/collama](https://raw.githubusercontent.com/5aharsh/collama/main/assets/banner.png)](https://github.com/5aharsh/collama)

This notebook demonstrates how to create a Strands agent with an open-source LLM served on Ollama inside a Colab instance . With this, you can run pretty much any small to medium sized models offerred by Ollama **for free**.


For the list of available models check [models being offerred by Ollama](https://ollama.com/library), including:
  

*   Qwen3
*   Phi4
*   Llama3
*   DeepSeek
*   Mistral
*   Minimax

## Before you proceed
---

Since by default the runtime type of Colab instance is CPU based, in order to use LLM models make sure to change your runtime type to T4 GPU (or better if you're a paid Colab user). This can be done by going to **Runtime > Change runtime type**.

While running your script be mindful of the resources you're using. This can be tracked at **Runtime > View resources**.

## Installing Ollama Dependencies
---

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`




In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,233 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,589 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

## Running Ollama
---

Service can be started by command `ollama serve`.



**OR** in python script, in order to use Ollama it needs to run as a service in background parallel to your scripts. Becasue Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.
`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [11]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

Download the qwen3:4b model to local storage.

In [12]:
!ollama pull qwen3:4b

# Build a Strands agent
---
To define an agent with the Strands Agents SDK, you define these three components in code:

1.   **Model**: Strands offers flexible model support. You can use any model in Amazon Bedrock that supports tool use and streaming, a model from Anthropic’s Claude model family through the Anthropic API, a model from the Llama model family via Llama API, Ollama for local development, and many other model providers such as OpenAI through LiteLLM.
2.   **Tools**: You can choose from thousands of published Model Context Protocol (MCP) servers to use as tools for your agent. Strands also provides 20+ pre-built example tools, including tools for manipulating files, making API requests, and interacting with AWS APIs. You can easily use any Python function as a tool, by simply using the Strands @tool decorator.
3.   **Prompt**: You provide a natural language prompt that defines the task for your agent, such as answering a question from an end user. You can also provide a system prompt that provides general instructions and desired behavior for the agent.

An agent interacts with its model and tools in a loop until it completes the task provided by the prompt.In each loop, Strands invokes the LLM with the prompt and agent context, along with a description of your agent’s tools. The LLM can choose to respond in natural language for the agent’s end user, plan out a series of steps, reflect on the agent’s previous steps, and/or select one or more tools to use. When the LLM selects a tool, Strands takes care of executing the tool and providing the result back to the LLM. When the LLM completes its task, Strands returns the agent’s final result.


![Core concepts of a Strands agent](https://d2908q01vomqb2.cloudfront.net/ca3512f4dfa95a03169c5a670a4c91a19b3077b4/2025/05/16/agentic-loop.png)


## Installing Strands Dependencies


In [4]:
!pip install strands-agents 'strands-agents[ollama]' strands-agents-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.8/312.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Create a Strands agent with Ollama model
Pull a supported ollama model(e.g. Qwen3-4B, one of the most powerful small language models) and serve at local host

In [13]:
from strands import Agent
from strands.models.ollama import OllamaModel
from IPython.display import Markdown

ollama_model = OllamaModel(
    host="http://localhost:11434",
    model_id="qwen3:4b",
    keep_alive="2h")

agent = Agent(model=ollama_model, callback_handler=None,
              system_prompt="""You are a helpful and friendly assistant named Strands.
              You answer user queries based on known information.
              Response with 'I don't know' if you are uncertain.""")


Invoke the agent for a response.

In [14]:
agent("Hello, who are you?")

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Hello! I'm Strands, a friendly AI assistant here to help with information and tasks. How can I assist you today? 😊"}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[7.162242889404297], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='f7405e69-cea7-4ec0-a1de-bc8707c584a7', usage={'inputTokens': 275, 'outputTokens': 56, 'totalTokens': 331})], usage={'inputTokens': 275, 'outputTokens': 56, 'totalTokens': 331})], traces=[<strands.telemetry.metrics.Trace object at 0x7ad326987950>], accumulated_usage={'inputTokens': 275, 'outputTokens': 56, 'totalTokens': 331}, accumulated_metrics={'latencyMs': 7111.170062}), state={}, interrupts=None, structured_output=None)

Create the agent with ollama model and start an loop for interactive conversation.

In [16]:
from IPython.display import Markdown
print("Strands Agent: Hi there! I'm Strands. How can I help you today?")
print("Type 'exit' or 'quit' to end the conversation.")

# Start the interactive loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Strands Agent: Goodbye!")
        break

    # Invoke the agent with the user's input
    try:
        response = str(agent(user_input))
        print(f"Strands Agent: {response}")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your chosen model is up running successfully.")


Strands Agent: Hi there! I'm Strands. How can I help you today?
Type 'exit' or 'quit' to end the conversation.
You: List the top use cases for AI agents 
Strands Agent: Great question! Since you asked again, I'll give you a **concise, focused list** of the top 5 most impactful AI agent use cases (based on current real-world adoption and scalability)—with minimal jargon so it's easy to scan:

1. **Personalized Customer Experience**  
   *Example:* Netflix/Amazon-style recommendations, dynamic pricing, or hyper-targeted marketing.

2. **Automated Customer Support**  
   *Example:* 24/7 chatbots handling 80%+ of routine queries (e.g., banks, e-commerce).

3. **Code & Content Generation**  
   *Example:* GitHub Copilot (code), ChatGPT (reports), DALL-E (images) — *no manual writing needed*.

4. **Real-Time Fraud Detection**  
   *Example:* Banks flagging suspicious transactions in milliseconds (e.g., PayPal, Chase).

5. **Supply Chain & Logistics Optimization**  
   *Example:* Walmart/UPS 

##Conversation management
In the Strands Agents SDK, context refers to the information provided to the agent for understanding and reasoning. This includes:
*   User messages
*   Agent responses
*   Tool usage and results
*   System prompts

As conversations grow, managing this context becomes increasingly important for several reasons:
*   Token Limits: Language models have fixed context windows (maximum tokens they can process)
*   Performance: Larger contexts require more processing time and resources
*   Relevance: Older messages may become less relevant to the current conversation
*   Coherence: Maintaining logical flow and preserving important information

Run the codes below and rerun the interactive conversation block above to have a conversation manager in place!

In [17]:
from strands.agent.conversation_manager import SummarizingConversationManager

conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=10,  # Always keep 10 most recent messages
)

agent = Agent(model=ollama_model, callback_handler=None,
              system_prompt="""You are a helpful and friendly assistant named Strands.
              You answer user queries based on known information.
              Response with 'I don't know' if you are uncertain.""",
              conversation_manager=conversation_manager)

## Add a tool to the agent


Create an account at tavily search and get a free key (1000 credits): https://app.tavily.com

In [18]:
import os
from google.colab import userdata
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [19]:
from strands_tools.tavily import tavily_search

agent = Agent(model=ollama_model, callback_handler=None,
              system_prompt="""You are a helpful and friendly assistant named Strands.
              You answer user queries based on known information and tools provided.
              Response with 'I don't know' if you are uncertain.""",
              tools=[tavily_search])

# The agent can now decide when to use the tavily_search tool autonomously
response = agent("What were the key findings of the latest AI safety research?")
print(response)

╭───────────────────────────────────────────── Tavily Search Results ─────────────────────────────────────────────╮
│ Query: key findings of the latest AI safety research                                                            │
│                                                                                                                 │
│ Results: 5 found                                                                                                │
│ --------------------------------------------------                                                              │
│                                                                                                                 │
│ [1] New study urges stronger AI safeguards to protect worker ...                                                │
│ URL:                                                                                                            │
│ https://www.msn.com/en-us/news/technology/new-study-urges-stronger-ai-safeguards-to-protect-worker-safety-and-w │
│ ell-being/ar-AA1TOFye                                                                                           │
│ Score: 0.68496037                                                                                               │
│ Content: Drawing on national and international data, the latest research identifies AI-related risks that       │
│ affect workplace dynamics and employee agency. It also                                                          │
│                                                                                                                 │
│                                                                                                                 │
│ [2] AI Advances Safety Through Reasoning Under Uncertainty ...                                                  │
│ URL: https://quantumzeitgeist.com/ai-advances-safety-reasoning-uncertainty-handling/                            │
│ Score: 0.650934                                                                                                 │
│ Content: The findings support the use of probabilistic methods in artificial intelligence, as modelling         │
│ uncertainty is fundamental to building reliable and safe systems.                                               │
│                                                                                                                 │
│                                                                                                                 │
│ [3] World 'may not have time' to prepare for AI safety risks ...                                                │
│ URL:                                                                                                            │
│ https://www.theguardian.com/technology/2026/jan/04/world-may-not-have-time-to-prepare-for-ai-safety-risks-says- │
│ leading-researcher                                                                                              │
│ Score: 0.44985238                                                                                               │
│ Content: AI safety expert David Dalrymple said rapid advances could outpace efforts to control powerful         │
│ systems. Dan Milmo Global technology editor.                                                                    │
│                                                                                                                 │
│                                                                                                                 │
│ [4] Building a safer future for AI research                                                                     │
│ URL: https://news.vt.edu/articles/2026/01/eng-coe-enged-secure-ai-research-engineering-education.html           │
│ Score: 0.43108177                                                                                               │
│ Content: “The risks of stolen intellectual property ca

Here are the key findings from the latest AI safety research, based on recent credible sources:

1. **Workplace Risks**: New studies highlight AI-related risks affecting workplace dynamics and employee agency, urging stronger safeguards to protect workers (MSN News).  
2. **Uncertainty Modeling**: Probabilistic methods are critical for building safe AI systems, as modeling uncertainty helps prevent unreliable or harmful outputs (Quantum Zeitgeist).  
3. **Time Constraints**: Experts warn that rapid AI advancements could outpace safety preparation efforts, leaving little time for effective risk mitigation (The Guardian).  
4. **Intellectual Property Threats**: Data collection and international AI collaborations pose risks of stolen intellectual property, requiring enhanced security protocols (Virginia Tech).  
5. **Policy Trade-offs**: Strict AI safety rules (e.g., refusal mechanisms for harmful content) reduce risky outputs but may inadvertently hinder competitive innovation in the AI 

# Optional: Strands agent with Bedrock backend
##Configure AWS Credentials
Bedrock connection requires AWS credentials. We'll use Colab's built-in secret manager to handle our keys securely.

1. Click on the **key icon** (Secrets) in the left sidebar.
2. Add the following three secrets:
   - `AWS_ACCESS_KEY_ID`
   - `AWS_SECRET_ACCESS_KEY`
   - `AWS_DEFAULT_REGION` (e.g., `us-east-1`)
3. Make sure to toggle the button to make the secret available in this notebook.

The following cell will then securely access these secrets and configure them as environment variables for our session.

In [20]:
import os
from google.colab import userdata

# Load AWS credentials from Colab secrets
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = userdata.get("AWS_DEFAULT_REGION")

Then we can connect agent with BedrockModel! Supported models in Bedrock:
https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html


In [21]:
from strands import Agent
from strands.models.bedrock import BedrockModel

# Use the US inference profile ID
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0"  # Note the "us." prefix
)
agent = Agent(model=model, callback_handler=None)

# Have a conversation with the agent
response = str(agent("Hello! Who are you?"))

# The agent automatically prints the response to the console
# The response is also captured in the variable
print(f"\nCaptured response: {response}")


Captured response: Hello! I'm Claude, an AI assistant created by Anthropic. I'm here to help with a wide variety of tasks like answering questions, helping with analysis and writing, math and coding, creative projects, and having conversations. Is there anything specific I can help you with today?

